In [1]:
from google.colab import files
import pandas as pd

# New Section

In [2]:
import nltk
from nltk.corpus import stopwords

In [3]:
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.3 MB/s eta 0:00:00


In [ ]:
!pip install textblob

In [ ]:
import textstat
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'textstat'

In [ ]:
!pip install wordfreq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
from wordfreq import word_frequency, zipf_frequency
from tqdm.notebook import tqdm

# Step 1: Upload fILES

In [ ]:
uploaded = files.upload()

Saving 3000words.csv to 3000words (2).csv


In [ ]:
df = pd.read_csv(filename, header=None, names=["Word"])
print("Full dataframe shape:", df.shape)


Full dataframe shape: (3000, 1)


### Step 1: Load CSV and Check Shape

In this step, we define a function `load_csv` that:
- Reads the CSV file into a pandas DataFrame.  
- Prints and returns its shape.  
- Stores the DataFrame in a global variable `df` for later use.  


In [4]:
# Upload file
uploaded = files.upload()

# Assign the uploaded file a name
filename = list(uploaded.keys())[0]

# Load into a DataFrame with a clear name
df = pd.read_csv(filename)

# Preview to confirm
df.head()
print("Full dataframe shape:", df.shape)

Saving words_analysis.csv to words_analysis.csv
Full dataframe shape: (2894, 74)


# Step 2: Load CSV (assuming one column of words)

In [ ]:
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename, header=None, names=["Word"])


# Step 3: Keep only the first 10 words

Remove stopwords

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df_no_stop = df[~df["Word"].str.lower().isin(stop_words)].reset_index(drop=True)
print("Shape after stopword removal:", df_no_stop.shape)

Shape after stopword removal: (2893, 1)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
for word in df['Word']:
    print(word)

a
abandon
ability
able
abortion
about
above
abroad
absence
absolute
absolutely
absorb
abuse
academic
accept
access
accident
accompany
accomplish
according
account
accurate
accuse
achieve
achievement
acid
acknowledge
acquire
across
act
action
active
activist
activity
actor
actress
actual
actually
ad
adapt
add
addition
additional
address
adequate
adjust
adjustment
administration
administrator
admire
admission
admit
adolescent
adopt
adult
advance
advanced
advantage
adventure
advertising
advice
advise
adviser
advocate
affair
affect
afford
afraid
African
African-American
after
afternoon
again
against
age
agency
agenda
agent
aggressive
ago
agree
agreement
agricultural
ah
ahead
aid
aide
AIDS
aim
air
aircraft
airline
airport
album
alcohol
alive
all
alliance
allow
ally
almost
alone
along
already
also
alter
alternative
although
always
AM
amazing
American
among
amount
analysis
analyst
analyze
ancient
and
anger
angle
angry
animal
anniversary
announce
annual
another
answer
anticipate
anxiety
any
an

# caluclaute repressiveness

In [ ]:
def wordnet_stats(word):
    synsets = wn.synsets(word)

    if not synsets:  # no entry in WordNet
        return {
            "Meanings": 0,
            "Num_Synonyms": 0,
            "Representativeness": 0
        }

    # number of senses = number of synsets
    meanings = len(synsets)

    # collect all synonyms across synsets
    synonyms = set()
    for syn in synsets:
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().lower())

    num_synonyms = len(synonyms)

    # representativeness = synonyms / synsets
    representativeness = num_synonyms / meanings if meanings > 0 else 0

    return {
        "Meanings": meanings,
        "Num_Synonyms": num_synonyms,
        "Representativeness": representativeness
    }

# Apply function to each word in df_no_stop
stats = df_no_stop["Word"].apply(wordnet_stats)

# Expand dictionary results into new dataframe columns
stats_df = pd.DataFrame(list(stats))

# Merge back into original dataframe
df_extended = pd.concat([df_no_stop, stats_df], axis=1)

df_extended

,Word,Meanings,Num_Synonyms,Representativeness
0,abandon,7,10,1.428571
1,ability,2,2,1.000000
2,able,4,3,0.750000
3,abortion,2,2,1.000000
4,abroad,4,3,0.750000
...,...,...,...,...
2888,yet,6,19,3.166667
2889,yield,17,28,1.647059
2890,young,14,20,1.428571
2891,youth,6,8,1.333333


In [ ]:
def wordnet_basic_stats(word):
    word = str(word).strip().lower()
    synsets = wn.synsets(word)
    meanings = len(synsets)
    synonyms = set()
    for syn in synsets:
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().lower())
    num_synonyms = len(synonyms)
    representativeness = num_synonyms / meanings if meanings > 0 else 0
    return pd.Series({
        "Meanings": meanings,
        "Num_Synonyms": num_synonyms,
        "Representativeness": representativeness
    })

# Apply to your cleaned dataframe
df_no_stop[['Meanings', 'Num_Synonyms', 'Representativeness']] = df_no_stop['Word'].apply(wordnet_basic_stats)


In [ ]:
print("Final dataframe shape:", df_no_stop.shape)
df_no_stop.head()

Final dataframe shape: (2893, 7)


,Word,Antonym_Count,Hyponym_Count,Hypernym_Count,Meanings,Num_Synonyms,Representativeness
0,abandon,0,18,19,7.0,10.0,1.428571
1,ability,1,40,4,2.0,2.0,1.000000
2,able,1,0,0,4.0,3.0,0.750000
3,abortion,0,4,4,2.0,2.0,1.000000
4,abroad,0,0,0,4.0,3.0,0.750000


from matplotlib import pyplot as plt
_df_0['Antonym_Count'].plot(kind='hist', bins=20, title='Antonym_Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Hyponym_Count'].plot(kind='hist', bins=20, title='Hyponym_Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Hypernym_Count'].plot(kind='hist', bins=20, title='Hypernym_Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['Word'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='Hyponym_Count', y='Word', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['Word'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='Hypernym_Count', y='Word', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['Word'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x='Meanings', y='Word', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
# Antonyms → list of antonyms (if available).

# Hyponyms → list of hyponyms (more specific words).

In [ ]:
def count_antonyms(word):
    word = str(word).strip().lower()
    antonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            for ant in lemma.antonyms():
                antonyms.add(ant.name().lower())
    return len(antonyms)

def count_hyponyms(word):
    word = str(word).strip().lower()
    hyponyms = set()
    for syn in wn.synsets(word):
        for hypo in syn.hyponyms():
            for lemma in hypo.lemmas():
                hyponyms.add(lemma.name().lower())
    return len(hyponyms)

def count_hypernyms(word):
    word = str(word).strip().lower()
    hypernyms = set()
    for syn in wn.synsets(word):
        for hyper in syn.hypernyms():
            for lemma in hyper.lemmas():
                hypernyms.add(lemma.name().lower())
    return len(hypernyms)

# Now apply to the entire dataframe!
df_no_stop['Antonym_Count'] = df_no_stop['Word'].apply(count_antonyms)
df_no_stop['Hyponym_Count'] = df_no_stop['Word'].apply(count_hyponyms)
df_no_stop['Hypernym_Count'] = df_no_stop['Word'].apply(count_hypernyms)

print("Final dataframe shape:", df_no_stop.shape)
df_no_stop.head()

Final dataframe shape: (2893, 4)


,Word,Antonym_Count,Hyponym_Count,Hypernym_Count
0,abandon,0,18,19
1,ability,1,40,4
2,able,1,0,0
3,abortion,0,4,4
4,abroad,0,0,0


In [ ]:
word = "abandon"
antonyms = set()
for syn in wn.synsets(word):
    for lemma in syn.lemmas():
        for ant in lemma.antonyms():
            antonyms.add(ant.name().lower())
print("abandon antonyms:", antonyms)


abandon antonyms: set()


In [ ]:
def find_antonyms(word):
    word_clean = str(word).strip().lower()
    antonyms = set()
    for syn in wn.synsets(word_clean):
        for lemma in syn.lemmas():
            for ant in lemma.antonyms():
                antonyms.add(ant.name().lower())
    return list(antonyms), len(antonyms)

# Choose your dataframe and column (replace as needed)
df_to_use = df_no_stop  # or df, if you want the full list
words = df_to_use["Word"].astype(str).tolist()

results = []
for word in words[:10]:  # Only first 10 for demo
    antonyms_list, antonym_count = find_antonyms(word)
    results.append({
        "Word": word,
        "Antonyms": antonyms_list,
        "Antonym_Count": antonym_count
    })


In [ ]:
for res in results:
    print(f"Word: {res['Word']}\nAntonyms: {res['Antonyms']}\nAntonym count: {res['Antonym_Count']}\n{'-'*40}")


Word: abandon
Antonyms: []
Antonym count: 0
----------------------------------------
Word: ability
Antonyms: ['inability']
Antonym count: 1
----------------------------------------
Word: able
Antonyms: ['unable']
Antonym count: 1
----------------------------------------
Word: abortion
Antonyms: []
Antonym count: 0
----------------------------------------
Word: abroad
Antonyms: []
Antonym count: 0
----------------------------------------
Word: absence
Antonyms: ['presence']
Antonym count: 1
----------------------------------------
Word: absolute
Antonyms: ['relative']
Antonym count: 1
----------------------------------------


In [ ]:
for syn in wn.synsets('abandon'):
    print(syn, [ant.name() for lemma in syn.lemmas() for ant in lemma.antonyms()])

Synset('abandon.n.01') []
Synset('wildness.n.01') []
Synset('abandon.v.01') []
Synset('abandon.v.02') []
Synset('vacate.v.02') []
Synset('abandon.v.04') []
Synset('abandon.v.05') []


# count hypernyms

In [ ]:
from nltk.corpus import wordnet as wn

def count_hypernyms(word):
    word_clean = str(word).strip().lower()
    hypernyms = set()
    for syn in wn.synsets(word_clean):
        for hyper in syn.hypernyms():
            for lemma in hyper.lemmas():
                hypernyms.add(lemma.name().lower())
    return len(hypernyms)

# Add column to your dataframe (use df_no_stop or your main df)
df_to_use = df_no_stop  # or df

df_to_use['Hypernym_Count'] = df_to_use['Word'].apply(count_hypernyms)

# Preview first 10 rows
df_to_use[['Word', 'Hypernym_Count']].head(10)


,Word,Hypernym_Count
0,abandon,19
1,ability,4
2,able,0
3,abortion,4
4,abroad,0
5,absence,9
6,absolute,2


Count hyponym

In [ ]:
def count_hyponyms(word):
    word_clean = str(word).strip().lower()
    hyponyms = set()
    for syn in wn.synsets(word_clean):
        for hypo in syn.hyponyms():
            for lemma in hypo.lemmas():
                hyponyms.add(lemma.name().lower())
    return len(hyponyms)

# Add column to your dataframe (use df_no_stop or your main df)
df_to_use['Hyponym_Count'] = df_to_use['Word'].apply(count_hyponyms)

# Preview first 10 rows
df_to_use[['Word', 'Hyponym_Count']].head(10)

,Word,Hyponym_Count
0,abandon,18
1,ability,40
2,able,0
3,abortion,4
4,abroad,0
5,absence,9
6,absolute,0


# Save

In [ ]:
print(df_no_stop.shape)


(7, 3)


In [ ]:
# Define your column order
cols = [
    "Word",
    "Meanings",
    "Num_Synonyms",
    "Representativeness",
    "Antonym_Count",
    "Hyponym_Count",
    "Hypernym_Count"
]

# Save to CSV in the specified order
df_no_stop[cols].to_csv('wordnet_analysis_output.csv', index=False)

# Download to your computer (if using Colab)
from google.colab import files
files.download('wordnet_analysis_output.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# look for a word

In [ ]:
# Ask for a word
word = input("Enter a word: ")

# Look up the word in WordNet
synsets = nltk.corpus.wordnet.synsets(word)

# Count the number of definitions, hypernyms, and hyponyms
num_defs = len(synsets)
num_hypernyms = len(synsets[0].hypernyms())
num_hyponyms = len(synsets[0].hyponyms())

# Print the results
print("Number of definitions:", num_defs)
print("Number of hypernyms:", num_hypernyms)
print("Number of hyponyms:", num_hyponyms)

# Frequency

In [ ]:
def get_wordfreqs(word):
    word_str = str(word).lower()
    freq = word_frequency(word_str, 'en')
    zipf = zipf_frequency(word_str, 'en')
    return pd.Series({'Word_Frequency': freq, 'Zipf_Frequency': zipf})

# Do NOT overwrite df! Just join the results
wordfreq_df = df['Word'].apply(get_wordfreqs)
df = pd.concat([df, wordfreq_df], axis=1)

# Preview all columns
print(df.columns)
df.head()


Index(['Word', 'Meanings', 'Num_Synonyms', 'Representativeness',
       'Antonym_Count', 'Hyponym_Count', 'Hypernym_Count', 'Word_Frequency',
       'Zipf_Frequency', 'Word_Frequency', 'Zipf_Frequency'],
      dtype='object')


,Word,Meanings,Num_Synonyms,Representativeness,Antonym_Count,Hyponym_Count,Hypernym_Count,Word_Frequency,Zipf_Frequency,Word_Frequency,Zipf_Frequency
0,abandon,7.0,10.0,1.428571,0,18,19,0.000010,4.02,0.000010,4.02
1,ability,2.0,2.0,1.000000,1,40,4,0.000091,4.96,0.000091,4.96
2,able,4.0,3.0,0.750000,1,0,0,0.000269,5.43,0.000269,5.43
3,abortion,2.0,2.0,1.000000,0,4,4,0.000017,4.24,0.000017,4.24
4,abroad,4.0,3.0,0.750000,0,0,0,0.000024,4.39,0.000024,4.39


In [ ]:

print("Final dataframe shape:", df.shape)

Final dataframe shape: (2893, 11)


# Ease of use

In [ ]:
import textstat
import numpy as np
import pandas as pd

def word_features(word):
    word_str = str(word)
    num_letters = len(word_str)
    num_syllables = textstat.syllable_count(word_str)

    inv_letters = 1/num_letters if num_letters > 0 else np.nan
    inv_syllables = 1/num_syllables if num_syllables > 0 else np.nan
    syll_per_letter = num_syllables/num_letters if num_letters > 0 else np.nan

    return pd.Series({
        'Num_Letters': num_letters,
        'Num_Syllables': num_syllables,
        'Inv_Num_Letters': inv_letters,
        'Inv_Num_Syllables': inv_syllables,
        'Syllables_per_Letter': syll_per_letter
    })

# Apply and concatenate without dropping existing columns
df = pd.concat([df, df['Word'].apply(word_features)], axis=1)

# Preview first 10 rows
df[['Word', 'Num_Letters', 'Num_Syllables',
    'Inv_Num_Letters', 'Inv_Num_Syllables', 'Syllables_per_Letter']].head(10)


,Word,Num_Letters,Num_Syllables,Inv_Num_Letters,Inv_Num_Syllables,Syllables_per_Letter
0,abandon,7.0,3.0,0.142857,0.333333,0.428571
1,ability,7.0,4.0,0.142857,0.250000,0.571429
2,able,4.0,2.0,0.250000,0.500000,0.500000
3,abortion,8.0,3.0,0.125000,0.333333,0.375000
4,abroad,6.0,2.0,0.166667,0.500000,0.333333
5,absence,7.0,2.0,0.142857,0.500000,0.285714
6,absolute,8.0,3.0,0.125000,0.333333,0.375000
7,absolutely,10.0,4.0,0.100000,0.250000,0.400000
8,absorb,6.0,2.0,0.166667,0.500000,0.333333
9,abuse,5.0,2.0,0.200000,0.500000,0.400000


# Flesh score

In [ ]:
import textstat
import numpy as np

def flesch_score(word):
    word_str = str(word)
    try:
        return textstat.flesch_reading_ease(word_str)
    except:
        return np.nan
df['Flesch_Reading_Ease'] = df['Word'].apply(flesch_score)

df[['Word', 'Flesch_Reading_Ease']].head(10)

,Word,Flesch_Reading_Ease
0,abandon,-47.98
1,ability,-132.58
2,able,36.62
3,abortion,-47.98
4,abroad,36.62
5,absence,36.62
6,absolute,-47.98
7,absolutely,-132.58
8,absorb,36.62
9,abuse,36.62


# Output

In [ ]:
# Save updated dataframe to CSV
df.to_csv('wordnet_analysis_output.csv', index=False)

# Download to your machine (if running in Colab)
from google.colab import files
files.download('wordnet_analysis_output.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Sentiment

In [ ]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def word_sentiment(word):
    word_str = str(word)
    scores = sia.polarity_scores(word_str)
    return pd.Series({
        'Sentiment_Positive': scores['pos'],
        'Sentiment_Negative': scores['neg'],
        'Polarity': scores['compound']
    })
df = pd.concat([df, df['Word'].apply(word_sentiment)], axis=1)

# Preview
df[['Word', 'Sentiment_Positive', 'Sentiment_Negative', 'Polarity']].head(10)

,Word,Sentiment_Positive,Sentiment_Negative,Polarity
0,abandon,0.0,1.0,-0.4404
1,ability,1.0,0.0,0.3182
2,able,0.0,0.0,0.0000
3,abortion,0.0,0.0,0.0000
4,abroad,0.0,0.0,0.0000
5,absence,0.0,0.0,0.0000
6,absolute,0.0,0.0,0.0000
7,absolutely,0.0,0.0,0.0000
8,absorb,0.0,0.0,0.0000
9,abuse,0.0,1.0,-0.6369


# Sentimet textblob

In [ ]:
from textblob import TextBlob
import pandas as pd

# Define the function (just the sentiment logic)
def blob_sentiment(word):
    word_str = str(word)
    blob = TextBlob(word_str)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        pos, neg = polarity, 0
    elif polarity < 0:
        pos, neg = 0, -polarity
    else:
        pos, neg = 0, 0

    return pd.Series({
        'Blob_Positive': pos,
        'Blob_Negative': neg,
        'Blob_Polarity': polarity
    })

# Apply function to dataframe (this must be OUTSIDE the function)
df = pd.concat([df, df['Word'].apply(blob_sentiment)], axis=1)

# Preview
df[['Word', 'Blob_Positive', 'Blob_Negative', 'Blob_Polarity']].head(10)


,Word,Blob_Positive,Blob_Negative,Blob_Polarity
0,abandon,0.0,0.0000,0.0000
1,ability,0.0,0.0000,0.0000
2,able,0.5,0.0000,0.5000
3,abortion,0.0,0.0000,0.0000
4,abroad,0.0,0.0000,0.0000
5,absence,0.0,0.0125,-0.0125
6,absolute,0.2,0.0000,0.2000
7,absolutely,0.2,0.0000,0.2000
8,absorb,0.0,0.0000,0.0000
9,abuse,0.0,0.0000,0.0000


# Setiwordnet

In [ ]:
# --- Imports ---
import nltk
import pandas as pd
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

# Download if missing
nltk.download('wordnet')
nltk.download('sentiwordnet')

# --- Define Function ---
def get_sentiwordnet_scores(word):
    word = str(word).lower().strip()
    pos_scores, neg_scores = [], []

    # Check across all POS categories
    for pos in [wn.NOUN, wn.VERB, wn.ADJ, wn.ADV]:
        for syn in wn.synsets(word, pos=pos):
            try:
                swn_syn = swn.senti_synset(syn.name())
                # Only add if sentiment exists
                if swn_syn.pos_score() > 0 or swn_syn.neg_score() > 0:
                    pos_scores.append(swn_syn.pos_score())
                    neg_scores.append(swn_syn.neg_score())
            except:
                continue

    if pos_scores or neg_scores:
        # Average scores
        avg_pos = sum(pos_scores)/len(pos_scores) if pos_scores else 0.0
        avg_neg = sum(neg_scores)/len(neg_scores) if neg_scores else 0.0
        avg_polarity = avg_pos - avg_neg

        # Maximum scores
        max_pos = max(pos_scores) if pos_scores else 0.0
        max_neg = max(neg_scores) if neg_scores else 0.0
        max_polarity = max_pos - max_neg
    else:
        avg_pos = avg_neg = avg_polarity = 0.0
        max_pos = max_neg = max_polarity = 0.0

    return pd.Series({
        'SWN_Positive_Avg': avg_pos,
        'SWN_Negative_Avg': avg_neg,
        'SWN_Polarity_Avg': avg_polarity,
        'SWN_Positive_Max': max_pos,
        'SWN_Negative_Max': max_neg,
        'SWN_Polarity_Max': max_polarity
    })

# --- Apply to your DataFrame ---
df = pd.concat([df, df['Word'].apply(get_sentiwordnet_scores)], axis=1)

# --- Preview ---
df[['Word',
    'SWN_Positive_Avg','SWN_Negative_Avg','SWN_Polarity_Avg',
    'SWN_Positive_Max','SWN_Negative_Max','SWN_Polarity_Max']].head(20)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


,Word,SWN_Positive_Avg,SWN_Positive_Avg,SWN_Negative_Avg,SWN_Negative_Avg,SWN_Polarity_Avg,SWN_Polarity_Avg,SWN_Positive_Max,SWN_Positive_Max,SWN_Negative_Max,SWN_Negative_Max,SWN_Polarity_Max,SWN_Polarity_Max
0,abandon,0.0,0.020833,0.0,0.250000,0.0,-0.229167,0.0,0.125,0.0,0.375,0.0,-0.250
1,ability,0.0,0.312500,0.0,0.062500,0.0,0.250000,0.0,0.500,0.0,0.125,0.0,0.375
2,able,0.0,0.156250,0.0,0.000000,0.0,0.156250,0.0,0.250,0.0,0.000,0.0,0.250
3,abortion,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000,0.0,0.000
4,abroad,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000,0.0,0.000
5,absence,0.0,0.125000,0.0,0.125000,0.0,0.000000,0.0,0.125,0.0,0.125,0.0,0.000
6,absolute,0.0,0.250000,0.0,0.312500,0.0,-0.062500,0.0,0.625,0.0,0.750,0.0,-0.125
7,absolutely,0.0,0.500000,0.0,0.000000,0.0,0.500000,0.0,0.500,0.0,0.000,0.0,0.500
8,absorb,0.0,0.250000,0.0,0.000000,0.0,0.250000,0.0,0.375,0.0,0.000,0.0,0.375
9,abuse,0.0,0.107143,0.0,0.482143,0.0,-0.375000,0.0,0.625,0.0,0.750,0.0,-0.125


In [ ]:
lysis

In [ ]:
d file

**bold text**

In [ ]:
variable_name = "IIIIII" # @param ["HE ","III","IIIIII"]
(VARIABLE_NAME)


NameError: name 'PRINT' is not defined

# Trying to find synanoms

### Step 2: Extract Words into a List

Define a function `extract_words` that:
- Reads the values in the column **`Word`** from the DataFrame.
- Converts them into a Python list.
- Returns both the list and its length.


In [5]:
def extract_words(df) -> tuple[list, int]:
    """
    Extracts all values from the 'Word' column into a list and returns the list and its length.

    Parameters:
        df (pd.DataFrame): Input DataFrame that must contain a 'Word' column.

    Returns:
        tuple: (list of words, length of the list)
    """
    words = df["Word"].dropna().tolist()  # dropna() removes missing values if any
    return words, len(words)

# Example usage
words, num_words = extract_words(df)
print("Number of words:", num_words)


Number of words: 2894


### Step 3: Group Words by Synonyms

Now that we already have a list called `words`:
- Use WordNet to find synonyms.
- Check which synonyms are also in our list.
- Assign each cluster a group ID.
- Build a table with columns: **word**, **group_id**.
- Return the DataFrame, total number of words, and total number of groups.


In [6]:
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd

# Ensure WordNet is downloaded (only needs to run once)
nltk.download('wordnet')
nltk.download('omw-1.4')

def group_synonyms(word_list: list[str]) -> tuple[pd.DataFrame, int, int]:
    """
    Groups words into synonym clusters based on WordNet.

    Parameters:
        word_list (list[str]): List of words to analyze.

    Returns:
        tuple: (DataFrame with columns ['word','group_id'],
                total number of words in the table,
                total number of unique groups)
    """
    groups = []          # list of sets, each set = group of synonyms
    word_to_group = {}   # dictionary mapping word → group_id

    for word in word_list:
        if word in word_to_group:
            continue  # already assigned to a group

        # Collect synonyms from WordNet that exist in our list
        synonyms = set()
        for syn in wn.synsets(word):
            for lemma in syn.lemmas():
                synonym = lemma.name().lower().replace("_", " ")
                if synonym in word_list:
                    synonyms.add(synonym)

        synonyms.add(word)  # always include the word itself

        # Merge with existing groups if overlap
        merged = False
        for group in groups:
            if not group.isdisjoint(synonyms):
                group.update(synonyms)
                merged = True
                break

        if not merged:
            groups.append(synonyms)

    # Map words → group_id
    for idx, group in enumerate(groups, start=1):
        for w in group:
            word_to_group[w] = idx

    # Build DataFrame
    df_groups = pd.DataFrame(list(word_to_group.items()), columns=["word", "group_id"])

    return df_groups, len(df_groups), len(groups)


# Use the words list we already created
df_groups, num_words, num_groups = group_synonyms(words)

print(df_groups.head())
print("Words in table:", num_words)
print("Number of groups:", num_groups)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


         word  group_id
0           1         1
1      secret         1
2  individual        49
3      single         1
4      person        49
Words in table: 2894
Number of groups: 1056


In [7]:
pd.set_option("display.max_rows", None)
print(df_groups)


                  word  group_id
0                    1         1
1               secret         1
2           individual        49
3               single         1
4               person        49
5              mystery         1
6                  one         1
7                 soul        49
8              private         1
9              someone        49
10            somebody        49
11           prominent        25
12              submit         2
13             publish         2
14          anticipate         2
15            sanction       160
16               crazy       221
17         communicate         2
18          background         2
19          appearance         2
20               relax         2
21          production         2
22               swear         2
23              partly         4
24                 big        25
25             inquiry         4
26         distinguish         2
27               phone         2
28          strengthen         2
29        

In [ ]:
## Finding synanoms

In [8]:
from nltk.corpus import wordnet as wn

tree_syns = wn.synsets("tree")
box_syns = wn.synsets("box")

print("Tree synsets:", [s.definition() for s in tree_syns])
print("Box synsets:", [s.definition() for s in box_syns])

overlap = set(lemma.name() for s in tree_syns for lemma in s.lemmas()) & \
          set(lemma.name() for s in box_syns for lemma in s.lemmas())

print("Overlap:", overlap)


Tree synsets: ['a tall perennial woody plant having a main trunk and branches forming a distinct elevated crown; includes both gymnosperms and angiosperms', 'a figure that branches from a single root', 'English actor and theatrical producer noted for his lavish productions of Shakespeare (1853-1917)', 'force a person or an animal into a position from which he cannot escape', 'plant with trees', 'chase an animal up a tree', 'stretch (a shoe) on a shoetree']
Box synsets: ['a (usually rectangular) container; may have a lid', 'private area in a theater or grandstand where a small group can watch the performance', 'the quantity contained in a box', 'a predicament from which a skillful or graceful escape is impossible', 'a rectangular drawing', 'evergreen shrubs or small trees', 'any one of several designated areas on a ball field where the batter or catcher or coaches are positioned', "the driver's seat on a coach", 'separate partitioned area in a public place for a few people', 'a blow wit

### Step 4: Save Synonym Groups to CSV

We will:
1. Save `df_groups` into a CSV file.
2. Use `google.colab.files.download` to download it to your computer.


In [9]:
from google.colab import files

# Save DataFrame to CSV
output_file = "synonym_groups.csv"
df_groups.to_csv(output_file, index=False)

# Download the file
files.download(output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Step 5: Filter Multi-Word Groups and Save

We will:
1. Count how many words are in each group.
2. Keep only groups where the count > 1.
3. Build a new DataFrame with columns **word** and **group_id**.
4. Save it as a CSV file for download.


In [10]:
from google.colab import files

# Step 1: count how many words per group
group_counts = df_groups["group_id"].value_counts()

# Step 2: find group_ids with more than one word
multi_groups = group_counts[group_counts > 1].index

# Step 3: filter df_groups
df_multi = df_groups[df_groups["group_id"].isin(multi_groups)]

# Step 4: save to CSV
output_file_multi = "multi_word_groups.csv"
df_multi.to_csv(output_file_multi, index=False)

# Download file
files.download(output_file_multi)

# Show preview in Colab
df_multi.head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,word,group_id
0,1,1
1,secret,1
2,individual,49
3,single,1
4,person,49


### Step 7: Expand Synonym Groups into Flat Table

We will:
1. For each word in the list, find its synonyms from WordNet.
2. Filter to keep only synonyms that also exist in our word list.
3. Expand into a flat table with columns:
   - **word** (the original word)
   - **synonym** (each synonym in its group, one per row)

This way, every word will have its own group, and synonyms may repeat across groups.


In [11]:
def build_flat_synonym_table(words):
    rows = []

    for word in words:
        # Get synonyms from WordNet
        synonyms = set()
        for syn in wn.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name().lower())

        # Keep only synonyms in our list
        valid_syns = sorted([w for w in synonyms if w in words])

        # Ensure the word itself is always included
        if word not in valid_syns:
            valid_syns.insert(0, word)

        # Expand to rows
        for syn in valid_syns:
            rows.append({"word": word, "synonym": syn})

    return pd.DataFrame(rows)

# Build the flat table
df_flat_synonyms = build_flat_synonym_table(words)

# Preview
df_flat_synonyms.head(20)


,word,synonym
0,one,1
1,one,one
2,one,single
3,like,care
4,like,like
5,like,similar
6,like,wish
7,time,clock
8,time,meter
9,time,sentence


In [12]:
# Save flat synonym table to CSV
output_file_synonyms = "flat_synonym_table.csv"
df_flat_synonyms.to_csv(output_file_synonyms, index=False)

# Download file
files.download(output_file_synonyms)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Step 10: Apply Multi-Criteria Stickiness Rule

We will:
1. Merge synonym pairs with their statistics.
2. Define "sticky" as satisfying at least **3 of 6 conditions**:
   - Meanings ≥ 9
   - Num_Synonyms ≥ 14
   - Zipf_Frequency > 5
   - Num_Letters ≤ 8
   - GPT_Polarity = 1
   - SWN_Polarity_Max_abs ≥ 0.25
3. Label each word (sticky or not).
4. Select synonym pairs where one word is sticky and the other is not.
5. Randomly sample 500 pairs.
6. Save them to a CSV file.


In [14]:
from google.colab import files
import pandas as pd

# Upload both files manually from your local machine
uploaded = files.upload()

# After upload, Colab will store them by filename
df_syn = pd.read_csv("flat_synonym_table.csv")
df_stats = pd.read_csv("words_analysis.csv")

print("Synonym table shape:", df_syn.shape)
print("Stats table shape:", df_stats.shape)


Saving flat_synonym_table.csv to flat_synonym_table (1).csv
Saving words_analysis.csv to words_analysis (1).csv
Synonym table shape: (10847, 2)
Stats table shape: (2894, 74)


In [20]:
from google.colab import files

# --- Stickiness Function ---
def is_sticky(row, prefix):
    """Check if a word/syn meets at least 3 of 6 stickiness conditions."""
    conditions = [
        row[f"{prefix}_meanings"] >= 9 if pd.notnull(row[f"{prefix}_meanings"]) else False,
        row[f"{prefix}_syns"] >= 14 if pd.notnull(row[f"{prefix}_syns"]) else False,
        row[f"{prefix}_zipf"] > 5 if pd.notnull(row[f"{prefix}_zipf"]) else False,
        row[f"{prefix}_letters"] <= 8 if pd.notnull(row[f"{prefix}_letters"]) else False,
        row[f"{prefix}_gptpol"] == 1 if pd.notnull(row[f"{prefix}_gptpol"]) else False,
        abs(row[f"{prefix}_swnpol"]) >= 0.25 if pd.notnull(row[f"{prefix}_swnpol"]) else False
    ]
    return sum(conditions) >= 3

# --- Convert Columns to Numeric ---
numeric_cols = [
    "word_meanings", "word_syns", "word_zipf", "word_letters", "word_gptpol", "word_swnpol",
    "syn_meanings", "syn_syns", "syn_zipf", "syn_letters", "syn_gptpol", "syn_swnpol"
]
for col in numeric_cols:
    df_merged[col] = pd.to_numeric(df_merged[col], errors="coerce")

# --- Apply Stickiness Rules ---
df_merged["word_sticky"] = df_merged.apply(lambda r: is_sticky(r, "word"), axis=1)
df_merged["syn_sticky"] = df_merged.apply(lambda r: is_sticky(r, "syn"), axis=1)

# --- Filter Pairs where One is Sticky, One is Not ---
df_pairs = df_merged[df_merged["word_sticky"] != df_merged["syn_sticky"]]

# --- Sample 500 Pairs ---
df_sample = df_pairs.sample(n=500, random_state=42)

# --- Save to CSV ---
output_file = "sticky_nonsticky_pairs.csv"
df_sample.to_csv(output_file, index=False)
files.download(output_file)

# --- Preview ---
df_sample[["word", "synonym", "word_sticky", "syn_sticky"]].head(20)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,word,synonym,word_sticky,syn_sticky
705,keep,proceed,True,False
2764,alone,entirely,True,False
10733,attach,bond,False,True
6213,ill,complaint,True,False
10832,depict,draw,False,True
8347,badly,seriously,True,False
3008,trust,desire,True,False
1853,form,phase,True,False
6834,remaining,continue,False,True
8974,contribute,bring,False,True


### Step X: Export Word–Synonym Stickiness Table

We will:
1. Select only the relevant columns from `df_sample`.
2. Save them as a CSV.
3. Download the file for inspection.


In [21]:
# Select only the needed columns
df_export = df_sample[["word", "synonym", "word_sticky", "syn_sticky"]]

# Save to CSV
output_file_pairs = "sticky_nonsticky_pairs.csv"
df_export.to_csv(output_file_pairs, index=False)

# Download the file
from google.colab import files
files.download(output_file_pairs)

# Show first rows for confirmation
df_export.head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,word,synonym,word_sticky,syn_sticky
705,keep,proceed,True,False
2764,alone,entirely,True,False
10733,attach,bond,False,True
6213,ill,complaint,True,False
10832,depict,draw,False,True


In [18]:
# Step 6: Save to CSV and download
output_file = "sticky_nonsticky_pairs.csv"
df_sample.to_csv(output_file, index=False)
files.download(output_file)

# Step 7: Preview in Colab
df_sample[["word", "synonym", "word_sticky", "syn_sticky"]].head(20)

NameError: name 'df_sample' is not defined